## Weekly Assignment-3: Delta Lake–based data storage on Databricks that:
1. Stores data efficiently on cloud storage.
2. Uses partitioning (partitionBy) to speed up queries.
3. Leverages Delta Lake features like:
    - ACID transactions
    - Time Travel
    - Schema evolution
    - Updates &amp; deletes
    - Optimize / Z-Ordering (optional bonus)
### Input Data Description
Assume you receive daily order data in this format (JSON or CSV):
- column type description
- order_id string Unique ID of the order
- order_timestamp timestamp When the order was placed (UTC)
- customer_id string Unique ID of the customer
country string Country code (e.g., &quot;US&quot;, &quot;IN&quot;)
- amount double Order total amount
currency string Currency code (e.g., &quot;USD&quot;, &quot;INR&quot;)
- status string &quot;CREATED&quot;, &quot;PAID&quot;, &quot;CANCELLED&quot;

Later, the business decides to add new fields:
- payment_method (e.g., &quot;CARD&quot;, &quot;UPI&quot;, &quot;COD&quot;)
- coupon_code (nullable string)

## Task-1: Ingest sample order data into a Spark DataFrame.

In [0]:
# /Volumes/main/default/my-volume/daily_orders_1000.csv
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, TimestampType

schema = StructType([
    StructField("order_id", StringType(), True),
    StructField("order_timestamp", TimestampType(), True),
    StructField("customer_id", StringType(), True),
    StructField("country", StringType(), True),
    StructField("amount", DoubleType(), True),
    StructField("currency", StringType(), True),
    StructField("status", StringType(), True)
])

# reading daily orders synthetic data
df = spark.read.format("csv").option("header", True).schema(schema).load("/Volumes/main/default/my-volume/daily_orders_1000.csv")

In [0]:
df.show() # displaying top 20 rows
df.count() # displaying number of rows

+----------+-------------------+-----------+-------+-------+--------+---------+
|  order_id|    order_timestamp|customer_id|country| amount|currency|   status|
+----------+-------------------+-----------+-------+-------+--------+---------+
|ORD-100000|2025-01-12 08:00:00|  CUST-1148|     AU| 222.34|     AUD|  CREATED|
|ORD-100001|2025-01-12 08:00:40|  CUST-9540|     GB| 136.47|     GBP|  CREATED|
|ORD-100002|2025-01-12 08:01:20|  CUST-8407|     DE|  99.99|     EUR|     PAID|
|ORD-100003|2025-01-12 08:02:00|  CUST-8267|     IN|1575.73|     INR|     PAID|
|ORD-100004|2025-01-12 08:02:40|  CUST-4861|     AU| 230.79|     AUD|CANCELLED|
|ORD-100005|2025-01-12 08:03:20|  CUST-1223|     CA|  349.6|     CAD|     PAID|
|ORD-100006|2025-01-12 08:04:00|  CUST-5203|     AU| 219.82|     AUD|  CREATED|
|ORD-100007|2025-01-12 08:04:40|  CUST-9386|     CA|  214.3|     CAD|     PAID|
|ORD-100008|2025-01-12 08:05:20|  CUST-3839|     AU| 204.18|     AUD|CANCELLED|
|ORD-100009|2025-01-12 08:06:00|  CUST-4

1000

## Task-2: Add a derived column order_date (date only from order_timestamp).

In [0]:

from pyspark.sql.functions import *

df = df.withColumn("order_date", to_date("order_timestamp"))
df.show(5) # showing top 5 rows 

+----------+-------------------+-----------+-------+-------+--------+---------+----------+
|  order_id|    order_timestamp|customer_id|country| amount|currency|   status|order_date|
+----------+-------------------+-----------+-------+-------+--------+---------+----------+
|ORD-100000|2025-01-12 08:00:00|  CUST-1148|     AU| 222.34|     AUD|  CREATED|2025-01-12|
|ORD-100001|2025-01-12 08:00:40|  CUST-9540|     GB| 136.47|     GBP|  CREATED|2025-01-12|
|ORD-100002|2025-01-12 08:01:20|  CUST-8407|     DE|  99.99|     EUR|     PAID|2025-01-12|
|ORD-100003|2025-01-12 08:02:00|  CUST-8267|     IN|1575.73|     INR|     PAID|2025-01-12|
|ORD-100004|2025-01-12 08:02:40|  CUST-4861|     AU| 230.79|     AUD|CANCELLED|2025-01-12|
+----------+-------------------+-----------+-------+-------+--------+---------+----------+
only showing top 5 rows


## Task-3: Write the DataFrame as a Delta table partitioned by country and order_date.

In [0]:
from delta.tables import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

# update schema
schema = StructType([
    StructField("order_id", StringType(), True),
    StructField("order_timestamp", TimestampType(), True),
    StructField("customer_id", StringType(), True),
    StructField("country", StringType(), True),
    StructField("amount", DoubleType(), True),
    StructField("currency", StringType(), True),
    StructField("status", StringType(), True),
    StructField("order_date", DateType(), True)     # add this
])

# create Delta table
(DeltaTable
    .create(spark)
    .tableName("main.default.orders_delta")
    .addColumns(schema)
    .partitionedBy("country", "order_date")
    .execute()
)

# load the Delta table
orders_delta = DeltaTable.forName(spark, "main.default.orders_delta")

# write data into the table
df.write.format("delta").mode("append").saveAsTable("main.default.orders_delta")


In [0]:
%sql
-- showing top 5 rows from table
SELECT * FROM main.default.orders_delta limit 5;



order_id,order_timestamp,customer_id,country,amount,currency,status,order_date
ORD-100000,2025-01-12T08:00:00.000Z,CUST-1148,AU,222.34,AUD,CREATED,2025-01-12
ORD-100004,2025-01-12T08:02:40.000Z,CUST-4861,AU,230.79,AUD,CANCELLED,2025-01-12
ORD-100006,2025-01-12T08:04:00.000Z,CUST-5203,AU,219.82,AUD,CREATED,2025-01-12
ORD-100008,2025-01-12T08:05:20.000Z,CUST-3839,AU,204.18,AUD,CANCELLED,2025-01-12
ORD-100010,2025-01-12T08:06:40.000Z,CUST-2467,AU,47.33,AUD,CREATED,2025-01-12


## Task-4: Verify the partition structure in the storage path.

In [0]:
%sql
SHOW PARTITIONS main.default.orders_delta;


country,order_date
AU,2025-01-12
GB,2025-01-12
CA,2025-01-12
DE,2025-01-12
IN,2025-01-12
US,2025-01-12


In [0]:
%sql
DESCRIBE DETAIL main.default.orders_delta;
-- observation: numFiles = 6 i.e. 6 partitions were created per country

format,id,name,description,location,createdAt,lastModified,partitionColumns,clusteringColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures,statistics,clusterByAuto
delta,0b3cb896-d578-4876-92a1-ea03158598b1,main.default.orders_delta,null,,2025-11-30T04:13:55.724Z,2025-11-30T04:14:02.000Z,"List(country, order_date)",List(),6,34590,"Map(delta.enableDeletionVectors -> true, delta.enableRowTracking -> true, delta.rowTracking.materializedRowIdColumnName -> _row-id-col-2f4590e9-90c8-476b-b261-ce3820fe9e1e, delta.rowTracking.materializedRowCommitVersionColumnName -> _row-commit-version-col-2566c474-c906-4e81-ba1c-7a9dcd7229e2)",3,7,"List(appendOnly, deletionVectors, domainMetadata, invariants, rowTracking)","Map(numRowsDeletedByDeletionVectors -> 0, numDeletionVectors -> 0)",false


## Task-5: Run queries that demonstrate partition pruning (e.g., filter on a single country and/or date).

In [0]:
%sql

SELECT *
FROM main.default.orders_delta
WHERE country = 'US';

order_id,order_timestamp,customer_id,country,amount,currency,status,order_date
ORD-100009,2025-01-12T08:06:00.000Z,CUST-4981,US,245.97,USD,CREATED,2025-01-12
ORD-100021,2025-01-12T08:14:00.000Z,CUST-4675,US,331.0,USD,PAID,2025-01-12
ORD-100028,2025-01-12T08:18:40.000Z,CUST-7226,US,316.07,USD,CREATED,2025-01-12
ORD-100030,2025-01-12T08:20:00.000Z,CUST-9363,US,87.24,USD,PAID,2025-01-12
ORD-100034,2025-01-12T08:22:40.000Z,CUST-8121,US,386.99,USD,PAID,2025-01-12
ORD-100038,2025-01-12T08:25:20.000Z,CUST-2745,US,248.97,USD,CANCELLED,2025-01-12
ORD-100048,2025-01-12T08:32:00.000Z,CUST-5646,US,82.03,USD,PAID,2025-01-12
ORD-100061,2025-01-12T08:40:40.000Z,CUST-5106,US,107.32,USD,PAID,2025-01-12
ORD-100063,2025-01-12T08:42:00.000Z,CUST-6671,US,54.12,USD,CREATED,2025-01-12
ORD-100074,2025-01-12T08:49:20.000Z,CUST-2439,US,76.72,USD,PAID,2025-01-12


In [0]:
%sql
EXPLAIN FORMATTED
SELECT *
FROM main.default.orders_delta
WHERE country = 'US';
-- displays partition filters indicating use of partition pruning

plan
"== Physical Plan == * ColumnarToRow (4) +- PhotonResultStage (3) +- PhotonProject (2) +- PhotonScan parquet main.default.orders_delta (1) (1) PhotonScan parquet main.default.orders_delta Output [8]: [order_id#12532, order_timestamp#12533, customer_id#12534, amount#12536, currency#12537, status#12538, country#12535, order_date#12539] Location: PreparedDeltaFileIndex [s3://dbstorage-prod-agfdu/uc/1f212cec-51df-430a-b621-cec5889adc83/a33e2734-9d3d-4e48-a1a9-7859526b7104/__unitystorage/catalogs/bdba4e0a-32d9-4b4f-8707-2c65df5d2842/tables/ed471415-25fd-4cc5-ae40-9ab04c0fa849] PartitionFilters: [isnotnull(country#12535), (country#12535 = US)] ReadSchema: struct (2) PhotonProject Input [8]: [order_id#12532, order_timestamp#12533, customer_id#12534, amount#12536, currency#12537, status#12538, country#12535, order_date#12539] Arguments: [order_id#12532, order_timestamp#12533, customer_id#12534, country#12535, amount#12536, currency#12537, status#12538, order_date#12539] (3) PhotonResultStage Input [8]: [order_id#12532, order_timestamp#12533, customer_id#12534, country#12535, amount#12536, currency#12537, status#12538, order_date#12539] (4) ColumnarToRow [codegen id : 1] Input [8]: [order_id#12532, order_timestamp#12533, customer_id#12534, country#12535, amount#12536, currency#12537, status#12538, order_date#12539] == Photon Explanation == The query is fully supported by Photon. == Optimizer Statistics (table names per statistics state) == missing = partial = orders_delta full = Corrective actions: consider running the following command on all tables with missing or partial statistics ANALYZE TABLE COMPUTE STATISTICS FOR ALL COLUMNS"


In [0]:
%sql
EXPLAIN FORMATTED
SELECT *
FROM main.default.orders_delta
WHERE country = 'CA' AND order_date = '2025-01-12';
-- observation : PartitionFilters: [isnotnull(order_date#12586), (order_date#12586 = 2025-01-12), isnotnull(country#12582), (country#12582 = CA)] 

plan
"== Physical Plan == * ColumnarToRow (4) +- PhotonResultStage (3) +- PhotonProject (2) +- PhotonScan parquet main.default.orders_delta (1) (1) PhotonScan parquet main.default.orders_delta Output [8]: [order_id#12579, order_timestamp#12580, customer_id#12581, amount#12583, currency#12584, status#12585, country#12582, order_date#12586] Location: PreparedDeltaFileIndex [s3://dbstorage-prod-agfdu/uc/1f212cec-51df-430a-b621-cec5889adc83/a33e2734-9d3d-4e48-a1a9-7859526b7104/__unitystorage/catalogs/bdba4e0a-32d9-4b4f-8707-2c65df5d2842/tables/ed471415-25fd-4cc5-ae40-9ab04c0fa849] PartitionFilters: [isnotnull(order_date#12586), (order_date#12586 = 2025-01-12), isnotnull(country#12582), (country#12582 = CA)] ReadSchema: struct (2) PhotonProject Input [8]: [order_id#12579, order_timestamp#12580, customer_id#12581, amount#12583, currency#12584, status#12585, country#12582, order_date#12586] Arguments: [order_id#12579, order_timestamp#12580, customer_id#12581, country#12582, amount#12583, currency#12584, status#12585, order_date#12586] (3) PhotonResultStage Input [8]: [order_id#12579, order_timestamp#12580, customer_id#12581, country#12582, amount#12583, currency#12584, status#12585, order_date#12586] (4) ColumnarToRow [codegen id : 1] Input [8]: [order_id#12579, order_timestamp#12580, customer_id#12581, country#12582, amount#12583, currency#12584, status#12585, order_date#12586] == Photon Explanation == The query is fully supported by Photon. == Optimizer Statistics (table names per statistics state) == missing = partial = orders_delta full = Corrective actions: consider running the following command on all tables with missing or partial statistics ANALYZE TABLE COMPUTE STATISTICS FOR ALL COLUMNS"


In [0]:
%sql
SELECT *
FROM main.default.orders_delta
WHERE country = 'IN' AND amount > 1000;

order_id,order_timestamp,customer_id,country,amount,currency,status,order_date
ORD-100003,2025-01-12T08:02:00.000Z,CUST-8267,IN,1575.73,INR,PAID,2025-01-12
ORD-100018,2025-01-12T08:12:00.000Z,CUST-6635,IN,1861.68,INR,PAID,2025-01-12
ORD-100019,2025-01-12T08:12:40.000Z,CUST-2086,IN,1611.49,INR,PAID,2025-01-12
ORD-100026,2025-01-12T08:17:20.000Z,CUST-7364,IN,1049.79,INR,PAID,2025-01-12
ORD-100033,2025-01-12T08:22:00.000Z,CUST-2379,IN,3345.0,INR,CANCELLED,2025-01-12
ORD-100051,2025-01-12T08:34:00.000Z,CUST-7616,IN,2197.92,INR,PAID,2025-01-12
ORD-100057,2025-01-12T08:38:00.000Z,CUST-9043,IN,1343.68,INR,PAID,2025-01-12
ORD-100068,2025-01-12T08:45:20.000Z,CUST-8582,IN,2194.56,INR,PAID,2025-01-12
ORD-100070,2025-01-12T08:46:40.000Z,CUST-5576,IN,2493.72,INR,PAID,2025-01-12
ORD-100092,2025-01-12T09:01:20.000Z,CUST-1882,IN,1068.92,INR,PAID,2025-01-12


In [0]:
%sql
EXPLAIN FORMATTED
SELECT *
FROM main.default.orders_delta
WHERE country = 'IN' AND amount > 1000;
-- observation : PartitionFilters: [isnotnull(country#12631), (country#12631 = IN)]

plan
"== Physical Plan == * ColumnarToRow (4) +- PhotonResultStage (3) +- PhotonProject (2) +- PhotonScan parquet main.default.orders_delta (1) (1) PhotonScan parquet main.default.orders_delta Output [8]: [order_id#12679, order_timestamp#12680, customer_id#12681, amount#12683, currency#12684, status#12685, country#12682, order_date#12686] DictionaryFilters: [(amount#12683 > 1000.0)] Location: PreparedDeltaFileIndex [s3://dbstorage-prod-agfdu/uc/1f212cec-51df-430a-b621-cec5889adc83/a33e2734-9d3d-4e48-a1a9-7859526b7104/__unitystorage/catalogs/bdba4e0a-32d9-4b4f-8707-2c65df5d2842/tables/ed471415-25fd-4cc5-ae40-9ab04c0fa849] PartitionFilters: [isnotnull(country#12682), (country#12682 = IN)] ReadSchema: struct RequiredDataFilters: [isnotnull(amount#12683), (amount#12683 > 1000.0)] (2) PhotonProject Input [8]: [order_id#12679, order_timestamp#12680, customer_id#12681, amount#12683, currency#12684, status#12685, country#12682, order_date#12686] Arguments: [order_id#12679, order_timestamp#12680, customer_id#12681, country#12682, amount#12683, currency#12684, status#12685, order_date#12686] (3) PhotonResultStage Input [8]: [order_id#12679, order_timestamp#12680, customer_id#12681, country#12682, amount#12683, currency#12684, status#12685, order_date#12686] (4) ColumnarToRow [codegen id : 1] Input [8]: [order_id#12679, order_timestamp#12680, customer_id#12681, country#12682, amount#12683, currency#12684, status#12685, order_date#12686] == Photon Explanation == The query is fully supported by Photon. == Optimizer Statistics (table names per statistics state) == missing = partial = orders_delta full = Corrective actions: consider running the following command on all tables with missing or partial statistics ANALYZE TABLE COMPUTE STATISTICS FOR ALL COLUMNS"


## Task-6: Demonstrate Delta Lake Time Travel: Write data, update some rows, then query older versions.

In [0]:
# check the current version (current version = 1)
spark.sql("DESCRIBE HISTORY main.default.orders_delta").show(truncate=False)


+-------+-------------------+----------------+-----------------------------+------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+------------------+------------------------+-----------+-----------------+-------------+---------------------------------------------------------------+------------+--------------------------------------------------+
|version|timestamp          |userId          |userName                     |operation   |operationParameters                                                                                                                                                        

In [0]:
# update data: mark all US orders as 'CANCELLED' 
# this should create a new version
spark.sql("""
UPDATE main.default.orders_delta
SET status = 'CANCELLED'
WHERE country = 'US'
""")

DataFrame[num_affected_rows: bigint]

In [0]:
# check the new verisons i.e 0, 1, 2
spark.sql("DESCRIBE HISTORY main.default.orders_delta").show(truncate=False)


+-------+-------------------+----------------+-----------------------------+------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+------------------+------------------------+-----------+-----------------+-------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+--------------------------------------------------+
|ver

In [0]:
# query the latest version to verify the update
spark.sql("""
SELECT *
FROM main.default.orders_delta
WHERE country = 'US'
""").show()
# observation : all status are 'CANCELLED'

+----------+-------------------+-----------+-------+------+--------+---------+----------+
|  order_id|    order_timestamp|customer_id|country|amount|currency|   status|order_date|
+----------+-------------------+-----------+-------+------+--------+---------+----------+
|ORD-100009|2025-01-12 08:06:00|  CUST-4981|     US|245.97|     USD|CANCELLED|2025-01-12|
|ORD-100021|2025-01-12 08:14:00|  CUST-4675|     US| 331.0|     USD|CANCELLED|2025-01-12|
|ORD-100028|2025-01-12 08:18:40|  CUST-7226|     US|316.07|     USD|CANCELLED|2025-01-12|
|ORD-100030|2025-01-12 08:20:00|  CUST-9363|     US| 87.24|     USD|CANCELLED|2025-01-12|
|ORD-100034|2025-01-12 08:22:40|  CUST-8121|     US|386.99|     USD|CANCELLED|2025-01-12|
|ORD-100038|2025-01-12 08:25:20|  CUST-2745|     US|248.97|     USD|CANCELLED|2025-01-12|
|ORD-100048|2025-01-12 08:32:00|  CUST-5646|     US| 82.03|     USD|CANCELLED|2025-01-12|
|ORD-100061|2025-01-12 08:40:40|  CUST-5106|     US|107.32|     USD|CANCELLED|2025-01-12|
|ORD-10006

In [0]:
# query the older version using VERSION AS OF (time travel)
spark.sql("""
SELECT *
FROM main.default.orders_delta VERSION AS OF 1
WHERE country = 'US'
""").show()
# observation - different statuses

+----------+-------------------+-----------+-------+------+--------+---------+----------+
|  order_id|    order_timestamp|customer_id|country|amount|currency|   status|order_date|
+----------+-------------------+-----------+-------+------+--------+---------+----------+
|ORD-100009|2025-01-12 08:06:00|  CUST-4981|     US|245.97|     USD|  CREATED|2025-01-12|
|ORD-100021|2025-01-12 08:14:00|  CUST-4675|     US| 331.0|     USD|     PAID|2025-01-12|
|ORD-100028|2025-01-12 08:18:40|  CUST-7226|     US|316.07|     USD|  CREATED|2025-01-12|
|ORD-100030|2025-01-12 08:20:00|  CUST-9363|     US| 87.24|     USD|     PAID|2025-01-12|
|ORD-100034|2025-01-12 08:22:40|  CUST-8121|     US|386.99|     USD|     PAID|2025-01-12|
|ORD-100038|2025-01-12 08:25:20|  CUST-2745|     US|248.97|     USD|CANCELLED|2025-01-12|
|ORD-100048|2025-01-12 08:32:00|  CUST-5646|     US| 82.03|     USD|     PAID|2025-01-12|
|ORD-100061|2025-01-12 08:40:40|  CUST-5106|     US|107.32|     USD|     PAID|2025-01-12|
|ORD-10006

## Task-7: Demonstrate Schema Evolution

In [0]:
# Demonstrate Schema Evolution:
from pyspark.sql.functions import lit
# Add payment_method & coupon_code to new data.
new_df = (
    df
    .withColumn("payment_method", lit("CARD"))      
    .withColumn("coupon_code", lit("NY2026"))          
)

# Write to the same Delta table, allowing schema evolution.
(new_df.write
    .format("delta")
    .mode("overwrite")
    .option("mergeSchema", "true")  
    .saveAsTable("main.default.orders_delta")
)
new_df.printSchema()


root
 |-- order_id: string (nullable = true)
 |-- order_timestamp: timestamp (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- country: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- currency: string (nullable = true)
 |-- status: string (nullable = true)
 |-- order_date: date (nullable = true)
 |-- payment_method: string (nullable = false)
 |-- coupon_code: string (nullable = false)



In [0]:
%sql
-- verifying that schema has evolved
DESCRIBE TABLE main.default.orders_delta;


col_name,data_type,comment
order_id,string,null
order_timestamp,timestamp,null
customer_id,string,null
country,string,null
amount,double,null
currency,string,null
status,string,null
order_date,date,null
payment_method,string,null
coupon_code,string,null


In [0]:
%sql
-- verify new data written correctly
SELECT order_id, payment_method, coupon_code
FROM main.default.orders_delta
ORDER BY order_id DESC
LIMIT 20;


order_id,payment_method,coupon_code
ORD-100999,CARD,NY2026
ORD-100998,CARD,NY2026
ORD-100997,CARD,NY2026
ORD-100996,CARD,NY2026
ORD-100995,CARD,NY2026
ORD-100994,CARD,NY2026
ORD-100993,CARD,NY2026
ORD-100992,CARD,NY2026
ORD-100991,CARD,NY2026
ORD-100990,CARD,NY2026


## Task-8: Demonstrate Updates & Deletes using Delta:

In [0]:
# Mark some orders as CANCELLED.
spark.sql("""
UPDATE main.default.orders_delta
SET status = 'CANCELLED'
WHERE country = 'US'
""")

# Delete orders below a certain amount (e.g., test data cleanup).
spark.sql("""
DELETE FROM main.default.orders_delta
WHERE amount < 100
""")


DataFrame[num_affected_rows: bigint]

In [0]:
%sql
-- verifying updates and deletes
DESCRIBE HISTORY main.default.orders_delta;


version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
10,2025-11-30T04:50:00.000Z,3598812822814195,sesharidhimapatnala@gmail.com,OPTIMIZE,"Map(predicate -> [""(((((('country <=> cast(AU as string)) AND ('order_date <=> cast(2025-01-12 as date))) OR (('country <=> cast(CA as string)) AND ('order_date <=> cast(2025-01-12 as date)))) OR (('country <=> cast(GB as string)) AND ('order_date <=> cast(2025-01-12 as date)))) OR (('country <=> cast(US as string)) AND ('order_date <=> cast(2025-01-12 as date)))) OR (('country <=> cast(DE as string)) AND ('order_date <=> cast(2025-01-12 as date))))""], auto -> true, clusterBy -> [], zOrderBy -> [], batchId -> 0)",null,List(3531838322673117),1130-035456-ls42y2gi-v2n,9,SnapshotIsolation,false,"Map(numRemovedFiles -> 1, numRemovedBytes -> 6328, p25FileSize -> 6362, numDeletionVectorsRemoved -> 1, minFileSize -> 6362, numAddedFiles -> 1, maxFileSize -> 6362, p75FileSize -> 6362, p50FileSize -> 6362, numAddedBytes -> 6362)",null,Databricks-Runtime/17.2.x-aarch64-photon-scala2.13
9,2025-11-30T04:49:58.000Z,3598812822814195,sesharidhimapatnala@gmail.com,DELETE,"Map(predicate -> [""(amount#16038 < 100.0)""])",null,List(3531838322673117),1130-035456-ls42y2gi-v2n,8,WriteSerializable,false,"Map(numRemovedFiles -> 0, numRemovedBytes -> 0, numCopiedRows -> 0, numDeletionVectorsAdded -> 5, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 2296, numDeletionVectorsUpdated -> 0, numDeletedRows -> 187, scanTimeMs -> 1451, numAddedFiles -> 0, numAddedBytes -> 0, rewriteTimeMs -> 834)",null,Databricks-Runtime/17.2.x-aarch64-photon-scala2.13
8,2025-11-30T04:49:54.000Z,3598812822814195,sesharidhimapatnala@gmail.com,UPDATE,"Map(predicate -> [""(country#15729 = US)""])",null,List(3531838322673117),1130-035456-ls42y2gi-v2n,7,WriteSerializable,false,"Map(numRemovedFiles -> 1, numRemovedBytes -> 5894, numCopiedRows -> 0, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 1066, numDeletionVectorsUpdated -> 0, scanTimeMs -> 7, numAddedFiles -> 1, numUpdatedRows -> 152, numAddedBytes -> 6588, rewriteTimeMs -> 1047)",null,Databricks-Runtime/17.2.x-aarch64-photon-scala2.13
7,2025-11-30T04:47:53.000Z,3598812822814195,sesharidhimapatnala@gmail.com,CREATE OR REPLACE TABLE AS SELECT,"Map(partitionBy -> [""country"",""order_date""], clusterBy -> [], description -> null, isManaged -> true, properties -> {""delta.enableDeletionVectors"":""true"",""delta.enableRowTracking"":""true"",""delta.rowTracking.materializedRowIdColumnName"":""_row-id-col-2f4590e9-90c8-476b-b261-ce3820fe9e1e"",""delta.rowTracking.materializedRowCommitVersionColumnName"":""_row-commit-version-col-2566c474-c906-4e81-ba1c-7a9dcd7229e2""}, statsOnLoad -> true)",null,List(3531838322673117),1130-035456-ls42y2gi-v2n,6,WriteSerializable,false,"Map(numFiles -> 6, numRemovedFiles -> 30, numRemovedBytes -> 182209, numDeletionVectorsRemoved -> 0, numOutputRows -> 1000, numOutputBytes -> 37452)",null,Databricks-Runtime/17.2.x-aarch64-photon-scala2.13
6,2025-11-30T04:47:37.000Z,3598812822814195,sesharidhimapatnala@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(3531838322673117),1130-035456-ls42y2gi-v2n,5,WriteSerializable,true,"Map(numFiles -> 6, numOutputRows -> 1000, numOutputBytes -> 37452)",null,Databricks-Runtime/17.2.x-aarch64-photon-scala2.13
5,2025-11-30T04:45:27.000Z,3598812822814195,sesharidhimapatnala@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(3531838322673117),1130-035456-ls42y2gi-v2n,4,WriteSerializable,true,"Map(numFiles -> 6, numOutputRows -> 1000, numOutputBytes -> 37452)",null,Databricks-Runtime/17.2.x-aarch64-photon-scala2.13
4,2025-11-30T04:44:27.000Z,3598812822814195,sesharidhimapatnala@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",nul

## Task-9: (Bonus) Optimize the table:

In [0]:
%sql
-- Use OPTIMIZE and optionally ZORDER on customer_id or order_date.

OPTIMIZE main.default.orders_delta;


path,metrics
,"List(4, 4, List(5934, 6397, 6251.25, 4, 25005), List(6304, 6588, 6445.0, 4, 25780), 6, null, null, 0, 1, 6, 2, true, 0, 0, 1764478363043, 1764478365088, 8, 4, null, List(4, 151), null, 10, 10, 1534, 0, null)"


In [0]:
%sql
--ZORDER by customer_id
OPTIMIZE main.default.orders_delta
ZORDER BY (customer_id);

path,metrics
,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 6, List(minCubeSize(107374182400), List(0, 0), List(6, 37405), 0, List(0, 0), 0, null), null, 0, 0, 6, 6, false, 0, 0, 1764478401909, 1764478402877, 8, 0, null, List(0, 0), null, 10, 10, 0, 0, null)"


In [0]:
%sql
-- order_date is a partition column. Z-Ordering can only be performed on data columns
-- ZORDER by currency
OPTIMIZE main.default.orders_delta
ZORDER BY (currency);

path,metrics
,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 6, List(minCubeSize(107374182400), List(0, 0), List(6, 37405), 0, List(0, 0), 0, null), null, 0, 0, 6, 6, false, 0, 0, 1764478471247, 1764478472054, 8, 0, null, List(0, 0), null, 10, 10, 0, 0, null)"


## Task-10: (Bonus) Show how small file problems can occur with too many partitions and how OPTIMIZE helps.

In [0]:
%sql
DESCRIBE DETAIL main.default.orders_delta;

/* observations: 
1. small-file problems occur when too many partitions create more physical folders than necessary leading to thousands of tiny files
2. if numFiles is very high relative to data volume, you have a small-file problem. 
3. OPTIMIZE compacts small files into few larger files, ZORDER speeds up queries due to data skipping
*/


format,id,name,description,location,createdAt,lastModified,partitionColumns,clusteringColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures,statistics,clusterByAuto
delta,0b3cb896-d578-4876-92a1-ea03158598b1,main.default.orders_delta,null,,2025-11-30T04:13:55.724Z,2025-11-30T04:52:45.000Z,"List(country, order_date)",List(),6,37405,"Map(delta.enableDeletionVectors -> true, delta.enableRowTracking -> true, delta.rowTracking.materializedRowIdColumnName -> _row-id-col-2f4590e9-90c8-476b-b261-ce3820fe9e1e, delta.rowTracking.materializedRowCommitVersionColumnName -> _row-commit-version-col-2566c474-c906-4e81-ba1c-7a9dcd7229e2)",3,7,"List(appendOnly, deletionVectors, domainMetadata, invariants, rowTracking)","Map(numRowsDeletedByDeletionVectors -> 0, numDeletionVectors -> 0)",false
